In [5]:
import json
from ckiptagger import construct_dictionary, WS, POS
ws = WS("./data")

In [6]:
with open('./youtube/醫療保健text.json') as f:
    data = json.loads(f.read())
with open('./stopword.txt', encoding='utf-8') as f:
    stop = f.read()

In [7]:
stopword = stop.split('\n')

In [8]:
# 使用分好詞的資料
alltext = []
for file in data.values():
    for name, texts in file.items():
        word_sentence_list = ws(list(texts))
        text_list = []
        for i in range(len(texts)):
            for word in word_sentence_list[i]:
                if word not in stopword:
                    text_list.append(word)
        if text_list != []:
            alltext.append(text_list)

In [9]:
# 測試用,所以先隨機給分類
import random

In [10]:
y = [str(random.randint(0,3)) for _ in range(len(alltext))]

In [11]:
# 形成tfidf模型
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

In [12]:
dct = Dictionary(alltext)

In [13]:
corpus = [dct.doc2bow(line) for line in alltext]

In [14]:
tfidf = TfidfModel(corpus)

In [23]:
# 存讀TfidfModel的方法
tfidf.save('tfidfmodel')
newtfm = TfidfModel.load('tfidfmodel')

In [15]:
import pandas as pd
# 整理出訓練的資料
TFIDF_matrix = tfidf[corpus]
traindata = pd.DataFrame(columns=[x for x in range(len(dct))])
for index, i in enumerate(TFIDF_matrix):
    for x in i:
        traindata.loc[index,x[0]] = x[1]
traindata = traindata.fillna(0)

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test= train_test_split(traindata,y,test_size=0.3,random_state=0)

In [19]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf = clf.fit(X_train, y_train)
y_pred=clf.predict(X_train)
y_test_pred = clf.predict(X_test)
print("多項分佈樸素貝葉斯，訓練樣本總數： %d 錯誤樣本數 : %d" % (X_train.shape[0],(y_train != y_pred).sum()))
print("多項分佈樸素貝葉斯，訓練樣本總數： %d 錯誤樣本數 : %d" % (X_test.shape[0],(y_test != y_test_pred).sum()))

多項分佈樸素貝葉斯，訓練樣本總數： 291 錯誤樣本數 : 3
多項分佈樸素貝葉斯，訓練樣本總數： 125 錯誤樣本數 : 86


In [28]:
from sklearn import metrics
print("訓練score",metrics.accuracy_score(y_train, y_pred))
print("測試score",metrics.accuracy_score(y_test, y_test_pred))

訓練score 0.9896907216494846
測試score 0.312


In [ ]:
# 存取sklearn的模型
import joblib
joblib.dump(clf, 'NB_model')
loaded_model = joblib.load('NB_model')

In [29]:
traindata.to_csv('traindata',index=False)